In [72]:
!pip install mistralai tqdm --quiet

from getdata import query_data, get_connection, execute_query
import pandas as pd 
import os
from mistralai import Mistral
from dotenv import load_dotenv
import tqdm

import json
load_dotenv()

True

In [33]:
query_data("SELECT * FROM inegalites_cinema.festival LIMIT 1")

Running query
Query run ended in 0.02 seconds


,id,nom,pays,ville,nom_du_prix,premiere_edition,mois,genre,description,link,created_at,updated_at
0,1,Festival de Cannes,France,Cannes,Palme d'Or,1946,Mai,Tous genres,Le Festival de Cannes est l'un des festivals d...,None,2025-02-14 12:38:13.382852,2025-02-14 12:38:13.382852


# CHARGEMENT DES DONNEES FESTIVAL

## LOAD DATASET liste_festivals_renommes.csv

UTILISATION DE MISTRAL AI POUR L'ENRICHISSMENT DES DONNEES

In [34]:
filename = 'data/liste_festivals_renommes.csv' 

df_festivals = pd.read_csv(filename)
df_festivals.head()

,festival_name,festival_link
0,FESTIVAL DE CANNES,https://mubi.com/fr/awards-and-festivals/canne...
1,VENICE FILM FESTIVAL,https://mubi.com/fr/awards-and-festivals/venic...
2,BERLIN INTERNATIONAL FILM FESTIVAL,https://mubi.com/fr/awards-and-festivals/berli...
3,ACADEMY AWARDS,https://mubi.com/fr/awards-and-festivals/oscar...
4,SUNDANCE FILM FESTIVAL,https://mubi.com/fr/awards-and-festivals/sunda...


In [35]:

MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
model = "mistral-large-latest"
client = Mistral(api_key=os.getenv("MISTRALAI_API_KEY"))

In [36]:
json_schema = """{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "type": "object",
    "properties": {
        "festivals": {
            "type": "array",
            "description": "Array of film festival information",
            "items": {
                "type": "object",
                "properties": {
                    "nom": {
                        "type": "string",
                        "description": "Name of the festival"
                    },
                    "pays": {
                        "type": "string",
                        "description": "Country where the festival takes place"
                    },
                    "ville": {
                        "type": "string",
                        "description": "City where the festival takes place"
                    },
                    "nom_du_prix": {
                        "type": "string",
                        "description": "Name of the main prize awarded at the festival"
                    },
                    "premiere_edition": {
                        "type": "integer",
                        "description": "Year of the first edition",
                    },
                    "mois": {
                        "type": "string",
                        "description": "Month(s) when the festival takes place"
                    },
                    "genre": {
                        "type": "string",
                        "description": "Main film genre or category focus of the festival"
                    },
                    "description": {
                        "type": "string",
                        "description": "Brief description of the festival"
                    }
                },
                "required": [
                    "nom",
                    "pays",
                    "ville",
                    "nom_du_prix",
                    "premiere_edition",
                    "mois",
                    "genre",
                    "description"
                ],
                "additionalProperties": true
            },
            "minItems": 1
        }
    },
    "required": ["festivals"],
    "additionalProperties": false
}"""

In [37]:
prompt = f"""
You will receive a list of the most famous film festivals in the world.

I want you to provide a json file with the maximum informations about each festival. 
The informations should include the name of the festival, the country, the city, the prize name, the first edition year, the month it usually happens, the genre, a short description  

Do not limit yourself to the informations provided in the list. If you consider that other festivals are not listed, add it. 
The result must be in french and in the format of the schema below.

<schema>
{json_schema}
</schema>
"""

prompt



'\nYou will receive a list of the most famous film festivals in the world.\n\nI want you to provide a json file with the maximum informations about each festival. \nThe informations should include the name of the festival, the country, the city, the prize name, the first edition year, the month it usually happens, the genre, a short description  \n\nDo not limit yourself to the informations provided in the list. If you consider that other festivals are not listed, add it. \nThe result must be in french and in the format of the schema below.\n\n<schema>\n{\n    "$schema": "https://json-schema.org/draft/2020-12/schema",\n    "type": "object",\n    "properties": {\n        "festivals": {\n            "type": "array",\n            "description": "Array of film festival information",\n            "items": {\n                "type": "object",\n                "properties": {\n                    "nom": {\n                        "type": "string",\n                        "description": "Name

In [38]:

# chat_response = client.chat.complete(
#     model=model,
#     messages=[
#         {"role": "system", "content": prompt},
#         {"role": "user", "content": ','.join(df_festivals["festival_name"].tolist())},
#     ],
#     response_format={"type": "json_object"},
# )


In [39]:
# import json
# json_festivals = json.loads(chat_response.choices[0].message.content)

In [40]:
# # LOAD THE JSON INTO A DATAFRAME
# df_festivals_llm = pd.DataFrame(json_festivals['festivals'])
# df_festivals_llm

In [41]:
# df_festivals_llm.to_sql('festival', con=get_connection(), schema='inegalites_cinema', if_exists='append', index=False)

# CHARGEMENT DES DONNEES FILMS

## LOAD DATASET Dataset5050_CNC Films Agréés 2003-2024.xlsx

In [42]:
# use openpyxl to get list of spreadsheet names 
filename = 'data/Dataset5050_CNC Films Agréés 2003-2024.xlsx'
xls = pd.ExcelFile(filename)
sheet_names = xls.sheet_names
print(f"{len(sheet_names)} feuilles trouvés dans le fichier {filename}")
sheet_names = [sheet for sheet in sheet_names if '20' in sheet]
sheet_names

22 feuilles trouvés dans le fichier data/Dataset5050_CNC Films Agréés 2003-2024.xlsx


['2023',
 '2022',
 '2021',
 '2020',
 '2019',
 '2018',
 '2017',
 '2016',
 '2015',
 '2014',
 '2013',
 '2012',
 '2011',
 '2010',
 '2009',
 '2008',
 '2007',
 '2006',
 '2005',
 '2004',
 '2003']

In [ ]:
all_realisateurs = []

for sheet in tqdm.tqdm(sheet_names, desc="Processing sheets"):
    df = pd.read_excel(filename, sheet_name=sheet, skiprows=4)
    
    realisateurs = [[name.strip() for name in tt.split('/')] for tt in df['REALISATEUR'].fillna("").tolist()]
    realisateurs = [item for sublist in realisateurs for item in sublist]
    
    all_realisateurs.extend(realisateurs)
    
all_realisateurs = list(set(all_realisateurs))
all_realisateurs.sort()

print(f"{len(all_realisateurs)} réalisateurs trouvés dans le fichier {filename}")

Processing sheets: 100%|██████████| 21/21 [00:06<00:00,  3.00it/s]

3499 réalisateurs trouvés dans le fichier data/Dataset5050_CNC Films Agréés 2003-2024.xlsx


In [44]:
MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
model = "mistral-large-latest"
client = Mistral(api_key=os.getenv("MISTRALAI_API_KEY"))

In [ ]:
json_schema = """{
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "type": "object",
    "properties": {
        "realisateurs": {
            "type": "array",
            "description": "Array of film festival directors",
            "items": {
                "type": "object",
                "properties": {
                    "nom": {
                        "type": "string",
                        "description": "Name of the director"
                    },
                    "pays": {
                        "type": "string",
                        "description": "Country of origin of the director"
                    },
                    "date_naissance": {
                        "type": "date",
                        "description": "Date of birth of the director"
                    },
                    "date_deces": {
                        "type": "date",
                        "description": "Date of death of the director"
                    },
                    "genre": {
                        "type": "string",
                        "description": "Gender of the director"
                    },
                    "film_genre" : {
                        "type": "string",
                        "description": "Main film genre of the director"
                    },
                },
                "required": [
                    "nom",
                    "pays",
                    "date_naissance",
                    "genre",
                    "film_genre"
                ],
                "additionalProperties": true
            },
            "minItems": 1
        }
    },
    "required": ["realisateurs"],
    "additionalProperties": false
}"""

In [46]:
prompt = f"""
You will receive a list of well known movie directors.

I want you to provide a json file with the maximum informations about each director. 
The informations should include the name of the director as indicated in the list, the country, the date of birth, the date of death if applicable, the gender and the main film genre.

If you consider that other directors are not listed, add them.
The result must be in french and in the format of the schema below.

<schema>
{json_schema}
</schema>
"""

In [55]:
def batch_process_realisateurs(list_realisateurs):
        
    chat_response = client.chat.complete(
        model=model,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": ','.join(list_realisateurs)},
        ],
        response_format={"type": "json_object"},
    )

    json_realisateurs = json.loads(chat_response.choices[0].message.content)
    return json_realisateurs['realisateurs']


In [ ]:
all_json_realisateurs = []

# # create loop to iterate over the list of directors in batches of 100
# for i in tqdm.tqdm(range(0, len(all_realisateurs), 100), desc="Processing directors in batches"):
#     all_json_realisateurs.extend(batch_process_realisateurs(all_realisateurs[i:i+100]))

Processing directors in batches:   0%|          | 0/35 [00:00<?, ?it/s]

Processing directors in batches: 100%|██████████| 35/35 [1:54:39<00:00, 196.57s/it]  


In [98]:
df_realisateurs = pd.DataFrame(all_json_realisateurs)

df_realisateurs['genre'] = df_realisateurs['genre'].map({
    'Masculin': 'Homme',
    'Féminin': 'Femme',
    'Homme': 'Homme',
    'Femme': 'Femme',
    'M': 'Homme',
    'F': 'Femme',
    'Inconnu': 'Inconnu'
})

df_realisateurs['date_naissance'] = pd.to_datetime(df_realisateurs['date_naissance'], errors='coerce')
df_realisateurs['date_deces'] = pd.to_datetime(df_realisateurs['date_deces'], errors='coerce')
df_realisateurs.to_csv('data/realisateurs.csv', index=False)

In [99]:
df_realisateurs

,nom,pays,date_naissance,genre,film_genre,date_deces
0,ABADI Sou,Maroc,1960-01-01,Homme,Drame,NaT
1,ABASSI Ali,Tunisie,1965-01-01,Homme,Documentaire,NaT
2,ABATUROV Alexander,Russie,1955-01-01,Homme,Drame,NaT
3,ABBASS Hiam,Liban,1960-01-01,Femme,Drame,NaT
4,ABBOU Karim,France,1970-01-01,Homme,Comédie,NaT
...,...,...,...,...,...,...
3929,ZOR Eitan,Israël,1975-01-01,Homme,Drame,NaT
3930,ZOUHANI Hakim,France,1981-01-01,Homme,Comédie,NaT
3931,ZUCHUAT Olivier,France,1975-01-01,Homme,Documentaire,NaT
3932,ZULAWSKI Andrzej,Pologne,1940-11-22,Homme,Drame,2016-02-17


In [100]:
df_realisateurs.genre.value_counts()

genre
Homme      3034
Femme       876
Inconnu      24
Name: count, dtype: int64

In [101]:
# array(['Masculin', 'Féminin', 'Homme', 'Femme', 'M', 'F', 'Inconnu'],
#       dtype=object)
# remap genre to Homme, Femme, Inconnu
df_realisateurs['genre'] = df_realisateurs['genre'].map({
    'Masculin': 'Homme',
    'Féminin': 'Femme',
    'Homme': 'Homme',
    'Femme': 'Femme',
    'M': 'Homme',
    'F': 'Femme',
    'Inconnu': 'Inconnu'
})
    

In [ ]:
# df_realisateurs.to_sql('realisateur', con=get_connection(), schema='inegalites_cinema', if_exists='append', index=False)

934

In [49]:
# Read both sheets from the Excel file
df_films = pd.read_excel('data/Dataset5050_César 1976-2024.xlsx', sheet_name='Films')
df_personnes = pd.read_excel('data/Dataset5050_César 1976-2024.xlsx', sheet_name='Personnes')


### PROCESS FILMS DATASET

In [50]:
df_films.head()

,Année,Méta,Catégorie,Nommés,Films,Gagnant,Duo,Sexe,Producteurs,Sexe_Prod
0,1976,Acteurs/actrices,Meilleure Actrice,Romy Schneider,L'IMPORTANT C'EST D'AIMER,G,NaN,F,NaN,NaN
1,1976,Acteurs/actrices,Meilleure Actrice,Isabelle Adjani,L'HISTOIRE D'ADÈLE H,NaN,NaN,F,NaN,NaN
2,1976,Acteurs/actrices,Meilleure Actrice,Catherine Deneuve,LE SAUVAGE,NaN,NaN,F,NaN,NaN
3,1976,Acteurs/actrices,Meilleure Actrice,Delphine Seyrig,INDIA SONG,NaN,NaN,F,NaN,NaN
4,1976,Acteurs/actrices,Meilleur Acteur,Philippe Noiret,LE VIEUX FUSIL,G,NaN,H,NaN,NaN


In [51]:
df_personnes.head()

,Année,Méta,Catégorie,Nommés,Prénoms,Noms,Films,Gagnant,Duo,Producteurs,Sexe,Naissance
0,1990,Technique,Meilleurs Décors,Michèle Abbe,Michèle,Abbe,BUNKER PALACE HÔTEL,NaN,NaN,NaN,F,0
1,1996,Technique,Meilleurs Décors,Michèle Abbe,Michèle,Abbe,MADAME BUTTERFLY,NaN,NaN,NaN,F,0
2,2022,Films,Meilleur Film Documentaire (CM),Giacomo Abbruzzese,Giacomo,Abbruzzese,AMERICA,NaN,NaN,NaN,H,1983
3,2009,Scénariste,Meilleur Scénario Original,Marc Abdelnour,Marc,Abdelnour,SÉRAPHINE,G,D,NaN,H,0
4,1999,Acteurs/actrices,Meilleur Espoir Masculin,Lionel Abelanski,Lionel,Abelanski,TRAIN DE VIE,NaN,NaN,NaN,H,1964


## LOAD DATASET Dataset5050_Cheffes de poste 2023.xlsx - LISTE_FINALE_FILMS_RETENUS.xlsx
